In [25]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch import flatten
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.datasets as datasets
from scipy.io import loadmat
from torch.optim.lr_scheduler import ReduceLROnPlateau, StepLR
import torch.optim.lr_scheduler as lr_scheduler

In [26]:
class FlowerClassifier(nn.Module):
    def __init__(self, num_classes=102):
        super(FlowerClassifier,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.conv4 = nn.Conv2d(in_channels=128, out_channels=256, kernel_size=3, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(256)
        self.conv5 = nn.Conv2d(in_channels=256, out_channels=512, kernel_size=3, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(512)
        self.pool = nn.MaxPool2d(kernel_size=2 ,stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512 * 7 * 7, 512)
        self.fc2 = nn.Linear(512, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool(self.relu(self.bn1(self.conv1(x))))
        x = self.pool(self.relu(self.bn2(self.conv2(x))))
        x = self.pool(self.relu(self.bn3(self.conv3(x))))
        x = self.pool(self.relu(self.bn4(self.conv4(x))))
        x = self.pool(self.relu(self.bn5(self.conv5(x))))

        x = self.flatten(x)  
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

In [27]:
model = FlowerClassifier()

In [28]:
trainTransform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])

In [29]:
batchSize = 64

testTransform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dataset = datasets.Flowers102(root='./data', split='train', transform=trainTransform, download=True)
val_dataset = datasets.Flowers102(root='./data', split='val', transform=testTransform, download=True)
test_dataset = datasets.Flowers102(root='./data', split='test', transform=testTransform, download=True)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=batchSize, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batchSize, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batchSize, shuffle=False)

In [30]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 20
best_val_acc = 0.0
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in train_loader:
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

    train_loss = running_loss / len(train_loader)
    train_acc = 100 * correct / total

    val_loss = 0.0
    val_correct = 0
    val_total = 0
    model.eval()
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item()
            _, predicted = outputs.max(1)
            val_total += labels.size(0)
            val_correct += predicted.eq(labels).sum().item()

    val_loss /= len(val_loader)
    val_acc = 100 * val_correct / val_total

    print("Epoch [" + str(epoch + 1) + "/" + str(num_epochs) + "]")
    print("Train Loss: {:.4f}, Train Accuracy: {:.2f}%".format(train_loss, train_acc))
    print("Validation Loss: {:.4f}, Validation Accuracy: {:.2f}%".format(val_loss, val_acc))
    
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), 'best_model.pth')

model.load_state_dict(torch.load('best_model.pth'))

Epoch [1/20]
Train Loss: 12.1491, Train Accuracy: 1.08%
Validation Loss: 6.1024, Validation Accuracy: 0.98%
Epoch [2/20]
Train Loss: 6.0360, Train Accuracy: 0.88%
Validation Loss: 5.0222, Validation Accuracy: 1.47%
Epoch [3/20]
Train Loss: 5.1145, Train Accuracy: 1.86%
Validation Loss: 4.7496, Validation Accuracy: 0.98%
Epoch [4/20]
Train Loss: 4.7583, Train Accuracy: 1.47%
Validation Loss: 4.5923, Validation Accuracy: 2.45%
Epoch [5/20]
Train Loss: 4.5867, Train Accuracy: 1.57%
Validation Loss: 4.4555, Validation Accuracy: 2.06%
Epoch [6/20]
Train Loss: 4.4991, Train Accuracy: 1.76%
Validation Loss: 4.3971, Validation Accuracy: 2.25%
Epoch [7/20]
Train Loss: 4.4039, Train Accuracy: 3.14%
Validation Loss: 4.3055, Validation Accuracy: 3.53%


In [ ]:
test_correct = 0
test_total = 0
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()

print(f'Accuracy on test set: {100 * correct / total:.2f}%')